# Transzfer-tanulás (60 + 30 pont)

**A notebookot importáld be a Colab rendszerbe, majd abban dolgozz!**

**Versenyző neve: [ÍRD IDE A NEVED]**

A modern mesterséges neurális hálók felé támasztott egyik elvárásunk, hogy képesek legyenek transzferálni az általuk már egy területen/feladaton elsajátított "tudást" más feladatok megoldására is. Például ha a hálónknak megtanítottuk már a kutyák és macskák megkülönböztetését, elvárhatjuk, hogy gyorsabban és kevesebb adatból is meg tudja már tanulni például a lovak és tigrisek megkülönböztetését. Ez számítási erőforrást és időt takarítana meg számunkra, kevés adaton is lehetővé tenné a tanítást, és  potenciálisan javíthatná a modellek teljesítményét a különböző feladatokon. Sajnos általánosan érvényes, garantáltan működő módszer a tudás-transzfer biztosítására nincs egyelőre, a transzfer-tanulás sikeressége adat- és modell-függő a legtöbbször. Ezekben a feladatokban azt fogjuk vizsgálni, hogy hogyan és mikor, mennyire sikeresen működik a transzfer tanulás. A transzfer tanulásról bővebben olvashattok [ebben a jegyzetben](https://cs231n.github.io/transfer-learning/), illetve [ebben a pytorch tutorialban](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html).


## Részfeladatok

Ez a feladat több részfeladatból áll, ezeket bármilyen sorrendben választhatod, de javasoljuk, hogy nagyjából sorban haladj.

Az utolsó feladat egy általatok választott rövid projekt, a feladat témakörén belül. Ebben a notebookban legfeljebb egy ilyet adhatsz be (de vannak ilyen miniprojektek más feladatokban)

Feladatok (`60 + 30`):
1. `5 pont` Modell-kiértékelés (pytorch, hibafüggvények)
1. `10 pont` Transzfer tanítás nélkül (pytorch, torchvision, pretrained models)
1. `10 pont` Beágyazás-vektorok kiszámolása (pytorch)
1. `5 pont` Adatbetöltés (pytorch, dataloaders)
1. `15 pont` Transzfer tanítás (pytorch, tanítás, matplotlib, értelmezés)
1. `15 pont` Modellek távolságának számítása (pytorch, matplotlib)
1. `30 pont` **Szabadon értelmezhető miniprojekt** (Te találod ki, mit csinálsz)


---
## Előkészületek


### Könyvtárak betöltése, beállítások

In [ ]:
import numpy as np
import os
import random
import torch

import matplotlib.pyplot as plt
import multiprocessing

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

A reprodukálható eredmények érdekében elengedhetetlen, hogy rögzítsük a véletlenszám generátor seed-jét.

In [ ]:
# for DL its critical to set the random seed so that students can have a
# baseline to compare their results to expected results.
# Read more here: https://pytorch.org/docs/stable/notes/randomness.html

# Call `set_seed` function in the exercises to ensure reproducibility.

def set_seed(seed=None, seed_torch=True):
    if seed is None:
        seed = np.random.choice(2 ** 32)
    random.seed(seed)
    np.random.seed(seed)
    if seed_torch:
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True

    print(f'A random seed értéke: {seed}')

# In case that `DataLoader` is used
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# inform the user if the notebook uses GPU or CPU.

def set_device():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device != "cuda":
        print("WARNING: A notebook  jelenleg nem használ GPU gyorsítást, "
              "ha lehetséges, a menüben a `Runtime` -> "
              "`Change runtime type` opciónál választ a `GPU`-t ")
    else:
        print("GPU használható a notebookban.")

    return device

set_seed(seed=2024)
device = set_device()

A random seed értéke: 2024


---
### A CIFAR-10 és CIFAR-100 adathalmaz betöltése

A CIFAR-10 és a CIFAR-100 is színes képet tartalmazó adathalmaz 50000 tanító és 10000 teszt képpel, ezek 32 x 32 pixelből állnak, és összesen 10, illetve 100 különböző kategóriából lehetnek, mindegyik kép rendelkezik címkével. A `torchvision.datasets.cifar.CIFAR` objetumon keresztül érhető el mindkét adathalmaz, onnan töltjük be őket, előre beállítjuk, hogy ne használjunk adataugmentálást és 256 kép legyen egy batch-nyi adat.

In [ ]:
def get_dataloaders(dataset_name, batch_size=256, augmentation=False):
    normalization_data = {'CIFAR100': ((0.5071, 0.4866, 0.4409), (0.2673, 0.2564, 0.2762)),
                          'CIFAR10': ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))}

    transform_train = transforms.Compose([])
    if augmentation:
        transform_train.transforms.append(transforms.RandomCrop(32, padding=4))
        transform_train.transforms.append(transforms.RandomHorizontalFlip())

    transform_train.transforms.append(transforms.ToTensor())
    transform_train.transforms.append(transforms.Normalize(mean=normalization_data[dataset_name][0],
                                                           std=normalization_data[dataset_name][1]))

    transform_test = transforms.Compose([transforms.ToTensor(),
                                         transforms.Normalize(mean=normalization_data[dataset_name][0],
                                                              std=normalization_data[dataset_name][1])])
    if dataset_name == 'CIFAR100':
        dataclass = torchvision.datasets.CIFAR100
    elif dataset_name == 'CIFAR10':
        dataclass = torchvision.datasets.CIFAR10

    trainset = dataclass(root=f'./{dataset_name}', train=True, download=True, transform=transform_train)
    testset = dataclass(root=f'./{dataset_name}', train=False, download=True, transform=transform_test)

    print(f"Objektum: {type(trainset)}")
    print(f"Tanító adatok shape-je: {trainset.data.shape}")
    print(f"Teszt adatok shape-je: {testset.data.shape}")
    print(f"Az osztályok száma: {np.unique(trainset.targets).shape[0]}")

    num_workers = multiprocessing.cpu_count()

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return trainloader, testloader


cifar100_trainloader, cifar100_testloader = get_dataloaders('CIFAR100')
cifar10_trainloader, cifar10_testloader = get_dataloaders('CIFAR10')

Files already downloaded and verified
Files already downloaded and verified
Objektum: <class 'torchvision.datasets.cifar.CIFAR100'>
Tanító adatok shape-je: (50000, 32, 32, 3)
Teszt adatok shape-je: (10000, 32, 32, 3)
Az osztályok száma: 100
Files already downloaded and verified
Files already downloaded and verified
Objektum: <class 'torchvision.datasets.cifar.CIFAR10'>
Tanító adatok shape-je: (50000, 32, 32, 3)
Teszt adatok shape-je: (10000, 32, 32, 3)
Az osztályok száma: 10


### Betanított ResNet modellek betöltése

A ResNet hálóarchitektúra igen népszerű és jó eredményeket elérő modellcsalád alapja, fő jellemzője, hogy reziduális blokkokból áll. A reziduális blokkokon belül jól ismert rétegtípusok, konvolúciós, pooling, bacth normalizáló rétegek találhatóak, és a blokk bemenetét a kimenetéhez adva egy speciális úgynevezett _shortcut_ vagy _skip connection_ is van. Az [eredeti cikkben](https://arxiv.org/abs/1512.03385) további részleteket olvashattok. A modellcsaládban különböző mélységű modelleket lehet konstruálni a blokkok számának növelésével, pl. ResNet-18, ResNet-32, ResNet-50. A ResNet-20 modell abban tér el ezektől, hogy míg az előbbiek inkább nagyobb méretű képek feldolgozására alkalmasak (pl. ImageNet), addig a ResNet-20 kisebb konvolúciós filterekkel operál, a CIFAR adathalmazok kisebb inputméretéhez alakítva. Az alábbi kód betölt a torchhub-ról [két betanított modellt, amelyekről további információt találtok ezen a github linken](https://github.com/chenyaofo/pytorch-cifar-models/).

In [ ]:
print(torch.hub.list("chenyaofo/pytorch-cifar-models", force_reload=True))

/usr/local/lib/python3.10/dist-packages/torch/hub.py:293: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or list(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use list(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/zipball/master" to /root/.cache/torch/hub/master.zip


['cifar100_mobilenetv2_x0_5', 'cifar100_mobilenetv2_x0_75', 'cifar100_mobilenetv2_x1_0', 'cifar100_mobilenetv2_x1_4', 'cifar100_repvgg_a0', 'cifar100_repvgg_a1', 'cifar100_repvgg_a2', 'cifar100_resnet20', 'cifar100_resnet32', 'cifar100_resnet44', 'cifar100_resnet56', 'cifar100_shufflenetv2_x0_5', 'cifar100_shufflenetv2_x1_0', 'cifar100_shufflenetv2_x1_5', 'cifar100_shufflenetv2_x2_0', 'cifar100_vgg11_bn', 'cifar100_vgg13_bn', 'cifar100_vgg16_bn', 'cifar100_vgg19_bn', 'cifar100_vit_b16', 'cifar100_vit_b32', 'cifar100_vit_h14', 'cifar100_vit_l16', 'cifar100_vit_l32', 'cifar10_mobilenetv2_x0_5', 'cifar10_mobilenetv2_x0_75', 'cifar10_mobilenetv2_x1_0', 'cifar10_mobilenetv2_x1_4', 'cifar10_repvgg_a0', 'cifar10_repvgg_a1', 'cifar10_repvgg_a2', 'cifar10_resnet20', 'cifar10_resnet32', 'cifar10_resnet44', 'cifar10_resnet56', 'cifar10_shufflenetv2_x0_5', 'cifar10_shufflenetv2_x1_0', 'cifar10_shufflenetv2_x1_5', 'cifar10_shufflenetv2_x2_0', 'cifar10_vgg11_bn', 'cifar10_vgg13_bn', 'cifar10_vgg16_b

In [ ]:
cifar10_model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet20", pretrained=True)
cifar100_model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar100_resnet20", pretrained=True)

Using cache found in /root/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master
Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/releases/download/resnet/cifar10_resnet20-4118986f.pt" to /root/.cache/torch/hub/checkpoints/cifar10_resnet20-4118986f.pt
100%|██████████| 1.09M/1.09M [00:00<00:00, 21.1MB/s]
Using cache found in /root/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master
Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/releases/download/resnet/cifar100_resnet20-23dac2f1.pt" to /root/.cache/torch/hub/checkpoints/cifar100_resnet20-23dac2f1.pt
100%|██████████| 1.11M/1.11M [00:00<00:00, 18.6MB/s]


In [ ]:
cifar10_model

CifarResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias

In [ ]:
cifar100_model

CifarResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias

### Beágyazás-vektorok letöltése

Hogy a számításokat leegyszerűsítsük, előre legeneráltuk nektek a fennebb betöltött CIFAR-100-on tanított modellel a CIFAR-10 tanító adathalmaz beágyazásvektorait, a modell utolsó-előtti rétegének kimeneteit, amelyeken sokkal gyorsabb már egy lineáris kimeneti réteget betanítani. [Ezek a beágyazás-vektorok itt elérhetőek](https://drive.google.com/file/d/1NU-TEU5w1Fhqm4NT6rmJbBspdI5QeMoA/view?usp=drive_link), a fájl egy (50000, 64, 1, 1) alakú és egy (50000) alakú tensort tartalmaz a beágyazásvektorokkal  és a nekik megfelelő címkékkel. Manuálisan is le lehet tölteni, de az alább megadott kód már be is tölti nektek tensor-okként ezeket.

In [ ]:
!wget 'https://drive.google.com/uc?id=1NU-TEU5w1Fhqm4NT6rmJbBspdI5QeMoA&export=download' -O /content/cifar10_embeddings_and_labels.pt

cifar10_embeddings, labels = torch.load('/content/cifar10_embeddings_and_labels.pt', map_location=torch.device('cpu'))

print('Beágyazás-vektorok tenzorának mérete: ', cifar10_embeddings.shape)
print('Címkék tenzorának mérete: ', labels.shape)

--2024-05-30 02:10:04--  https://drive.google.com/uc?id=1NU-TEU5w1Fhqm4NT6rmJbBspdI5QeMoA&export=download
Resolving drive.google.com (drive.google.com)... 172.217.214.102, 172.217.214.138, 172.217.214.100, ...
Connecting to drive.google.com (drive.google.com)|172.217.214.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1NU-TEU5w1Fhqm4NT6rmJbBspdI5QeMoA&export=download [following]
--2024-05-30 02:10:04--  https://drive.usercontent.google.com/download?id=1NU-TEU5w1Fhqm4NT6rmJbBspdI5QeMoA&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.193.132, 2607:f8b0:4001:c0f::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.193.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13201806 (13M) [application/octet-stream]
Saving to: ‘/content/cifar10_embeddings_and_labels.pt’

/content/cifar10_em 100%[======

---
## **1\. Feladat - Modell-kiértékelés (5 pont)**

Implementálj egy hasznos segédfüggvényt, amely paraméterként egy pytorch modellt és egy adatloadert kap, és azt csinálja, hogy az adatloader által betöltött adatokon osztályozás szempontjából kiértékeli a modellt: kiszámolja az átlagos pontosságot és az átlagos keresztentrópia hibafüggvény-értéket, ezt a két számot téríti vissza. Figyelj arra, hogy kiértékeléskor be kell állítani, hogy a modelt evaluáljuk csak, nem tanítjuk (ennek függvényében például változik a BatchNorm működése is), illetve arra, hogy gradiensszámolás ilyenkor ne történejen semmiképp. Teszteld a függvényed egy már betöltött modellel és tesztadat-loaderrel.

### A megoldásod

Használj kód- és szövegcellákat szükség szerint.

## **2\. Feladat - Transzfer tanítás előtt (10 pont)**


Töltsd be két ResNet-20 modellt: az egyik legyen a CIFAR-100 adathalmazon osztályozásra betanítva, a másik pedig a CIFAR-10 adathalmazon, ehhez találsz segítséget a megadott kódban, illetve [a github oldalon](https://github.com/chenyaofo/pytorch-cifar-models/), ahonnan a betöltött modellek származnak. Írd le, milyen pontosságot érnek el a megfelelő teszt-halmazokon. Cseréld ki a CIFAR-100 adaton tanított modell utolsó rétegét a CIFAR-10 adaton tanított modellével. Számold ki az így kapott háló pontosságát a CIFAR-10 teszt-halmazán, és hasonlítsd össze egy véletlenszerűen inicializált megfelelő ResNet-20 háló CIFAR-10 teszt-halmazon mért teljesítményével, hogy kiderüljön, történik-e a tudás-transzfer továbbtanítás nélkül a CIFAR-100-ról. Fogalmazd meg szövegesen a kapott eredményeket.


### A megoldásod

## **3\. Feladat - Beágyazás-vektorok (10 pont)**

A transzfer tanulás során a forrás-adathalmazon (esetünkben ez a CIFAR-100 lesz) betanított modellt - tipikusan elegendő annak csak az utolsó rétegét - a cél-adaton (esetünkben a CIFAR-10) tanítjuk, úgymond finomhangoljuk. Hogy a számításokat leegyszerűsítsük, előre legeneráltuk nektek a fennebb betöltött CIFAR-100-on tanított modellel a CIFAR-10 tanító adathalmaz beágyazásvektorait, a modell utolsó-előtti rétegének kimeneteit, amelyeken sokkal gyorsabb már egy lineáris kimeneti réteget betanítani. [Ezek a beágyazás-vektorok itt elérhetőek](https://drive.google.com/file/d/1NU-TEU5w1Fhqm4NT6rmJbBspdI5QeMoA/view?usp=drive_link), a fájl egy (50000, 64, 1, 1) alakú és egy (50000) alakú tensort tartalmaz a beágyazásvektorokkal  és a nekik megfelelő címkékkel. Manuálisan is le lehet tölteni, de az alább megadott kód már be is tölti nektek tensor-okként ezeket. Egy batch-nyi adaton számold ki te is ezeket a beágyazás-vektorokat és ellenőrizd, hogy ezek megegyeznek az általunk megadottakkal, tudva, hogy a megadott kódban a random seed rögzítése miatt a megadott adatloader ugyanabban a sorrendben tölti be az adatokat, mint ahogyan a beágyazásvektorok számításakor mi is használtuk, és nem engedtük, hogy az adatbetöltő összekeverje az adatokat (azaz a shuffle paraméterét False-ra állítottuk). Elegendő, ha az abszolút eltérés közöttük kevesebb, mint 0.0001.

### A megoldásod

## **4\. Feladat - Adatbetöltés (5 pont)**

A megadott beágyazásvektorokhoz társítsd hozzá a megfelelő osztálycímkéket, tudva, hogy a megadott kódban a random seed rögzítése miatt a megadott adatloader ugyanabban a sorrendben tölti be az adatokat, mint ahogyan a beágyazásvektorok számításakor mi is használtuk (tehát a beágyazásvektorok ugyanabban a sorrendben vannak megadva nektek). Állíts elő egy adatloadert, ami már a beágyazásvektorokon való tanításhoz tölti be batch-enként az adatokat.

### A megoldásod

## **5\. Feladat - Transzfer tanítás (15 pont)**

Alkalmasan megválasztott paraméterekkel taníts az előző feladatban megadott beágyazásvektorokon egy egyetlen lineáris rétegből álló modellt a CIFAR-10 adathalmazon osztályozásra. Ez a CIFAR-100-on tanított modell CIFAR-10-en való transzfer tanításának felel meg. A tanítás után a CIFAR100-on betanított modell kimeneti rétegét cseréld le az újonnan betanított lineáris rétegre. Ha már sikerül jól betanítani a lineáris modellt, ábrázold egy ábrán a tanítási epochok számának függvényében, legalább 5 epochon keresztül, hogyan változik a lineáris réteg tanítása során a módosított háló pontossága a CIFAR-10 teszt-halmazán.

### A megoldásod

## **6\. Feladat - Modellek távolsága (15 pont)**

Számold ki három modell paramétereinek a páronkénti $L_2$-távolságát. A három összehasonlított modell legyen a CIFAR-10 osztályozására véletlenszerűen inicializiált, a CIFAR-10-en betanított modell, és a CIFAR-100-on tanított modell az 5. feladatban betanított lineáris rétegre lecserélt kimeneti réteggel - részpontszámért ez lehet a CIFAR-10-en betanított modell kimeneti rétegére cserélt is. Jelenítsd meg képként a páronkénti távolságokat egy 3x3 alakú mátrixban, amelynek a cellái az távolság-értékek alapján vannak beszínezve. Fogalmazd meg szövegesen, hogyan értelmezed a kapott értékeket.

### A megoldásod

## 7\. Feladat -  Miniprojekt - önálló exploráció (30 pont)**

Ha a többi részfeladatot nagyrészt megoldottad, most rád bízzuk, hogy milyen kiegészítéseket szertnél még implementálni a témával kapcsolatban. Itt van néhány példa:

* Speciális hibafüggvény implementálása transzfer tanuláshoz
* Transzfer tanítás szürkeáryalatos képi adathalmazra
* Vizualizáció például t-SNE segítségével, mennyire klasztereződnek a CIFAR-10 megadott beágyazásvektorai

Minden miniprojektért (ebben a notebookban csak egyet adhatsz be, de az NLP-4, CV-4 notebookokban is lehet) max *30 pont* jár. Egy 30 pontos projekt jellemzői:
* **motiváció és kreativitás:** érdekes, kreatív projekt, jó motiváció, hipotézisek-kérdések-célok egyértelmű leírása, referenciák az irodalomra, ahol releváns
* **implementáció:** nem-triviális munka, ami demonstrálja a technikai felkészültséget, helyes implementáció, jó eszközválasztás, kompakt és olvasható kód
* **kiértékelés és konklúziók:** fair és részletes kiértékelés, ábrák kiváló használata (feliratok, címkék, stb.), tömör de célratörő konklúzió.